In [ ]:
import pathlib

In [ ]:
# Makes it so any changes in pymedphys is automatically
# propagated into the notebook without needing a kernel reset.
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
from pymedphys._data import zenodo, upload
from pymedphys.labs.autosegmentation import indexing, filtering

In [ ]:
data_path_root = pathlib.Path.home().joinpath('.data/dicom-ct-and-structures')

name_mappings_path = data_path_root.joinpath('name_mappings.json')
names_map = filtering.load_names_mapping(name_mappings_path)

In [ ]:
(
    ct_image_paths,
    structure_set_paths,
    ct_uid_to_structure_uid,
    structure_uid_to_ct_uids,
) = indexing.get_uid_cache(data_path_root)

In [ ]:
(
    structure_names_by_ct_uid,
    structure_names_by_structure_set_uid,
) = indexing.get_cached_structure_names_by_uids(
    data_path_root, structure_set_paths, names_map
)

In [ ]:
stucture_uids = structure_uid_to_ct_uids.keys()

In [ ]:
prep_for_zenodo_path = pathlib.Path().home().joinpath('Documents', 'prep-for-zenodo')
zip_paths = list(prep_for_zenodo_path.glob('*'))

len(zip_paths)

In [ ]:
def get_path(uid, zip_paths):
    filtered_path = [path for path in zip_paths if uid in path.name]
    assert len(filtered_path) == 1
    path = filtered_path[0]
    
    return path

In [ ]:
author = 'PyMedPhys Contributors'
use_sandbox = False

for structure_uid in stucture_uids:
    title = f'auto-segmentation-{structure_uid}'
    
    filepaths = [get_path(structure_uid, zip_paths)]
    
    ct_uids = structure_uid_to_ct_uids[structure_uid]
    for ct_uid in ct_uids:
        filepaths.append(get_path(ct_uid, zip_paths))
        
    print(len(set(filepaths)))
    print(len(filepaths))
    assert len(set(filepaths)) == len(filepaths)
        
    upload.upload_files_to_zenodo(filepaths, title, author, use_sandbox=use_sandbox)